In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number, lit
from pyspark.sql.window import Window



catalog = "fingrid_test_workspace"

schema = "fingrid_silver"

table_name = "electricity_consumption"

table = ".".join([catalog, schema, table_name])

df = spark.read.table(table)


df = df.select("customer_type","time_series_type","res").dropDuplicates()

w = Window().orderBy(lit(1))
df = df.withColumn('id', row_number().over(w).cast('bigint'))

df = df.select(
    "id", "customer_type", "time_series_type", "res"
)

df = df.withColumnRenamed("id", "customerID")

df.display()

In [0]:
external_path = "abfss://gold@fingridtest.dfs.core.windows.net/gold/dim_customer/"
df.write.format("delta").mode("overwrite").save(external_path)



In [0]:
%sql
CREATE TABLE IF NOT EXISTS fingrid_test_workspace.fingrid_gold.dim_customer
USING DELTA
LOCATION 'abfss://gold@fingridtest.dfs.core.windows.net/gold/dim_customer/'


In [0]:
%sql
select * from fingrid_test_workspace.fingrid_gold.dim_customer